In [ ]:
import os
from os.path import join, isdir, isfile, abspath, dirname, splitext, basename, split
from parse import parse
import math

from IPython.display import Markdown, display
from dotenv import load_dotenv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from misc_util.pretty_print import pretty_print as print
from derive_conceptualspace.pipeline import CustomContext, SnakeContext
from derive_conceptualspace.settings import ENV_PREFIX, get_setting
from derive_conceptualspace import settings

plt.rcParams['figure.figsize'] = [16, 10]

In [ ]:
assert load_dotenv(abspath(join(os.getcwd(), "..", "..", "config", "_select_env.env")))
assert load_dotenv(os.environ["MA_ENV_FILE"])

candidates = [join(path, name)[len(os.environ["MA_BASE_DIR"]):] for path, subdirs, files in os.walk(join(os.environ["MA_BASE_DIR"], "")) for
              name in files if name.startswith("clusters.json")]
candidates = [i for i in candidates if i.startswith(os.environ["MA_DATASET"]) and not "backup" in i.lower()]
print(f"There are {len(candidates)} different parameter-combis:")
# display(candidates)
configs = [parse(os.sep.join(settings.DIR_STRUCT+["clusters.json"]), cand).named for cand in candidates]
print_cnf = {k: list(set(dic[k] for dic in configs)) for k in configs[0]}
print_cnf = {k: v[0] if len(v) == 1 else v for k, v in print_cnf.items()}
print_cnf

In [ ]:
def show_metrics(metlist):
    metrics = list(list(metlist.values())[0].keys())
    w, h = math.ceil(math.sqrt(len(metrics))), math.ceil(math.sqrt(len(metrics)))
    for n, met in enumerate(metrics):
        plt.subplot(w, h, n+1)
        vals = [i[met] for i in metlist.values()]
        #vals = (np.digitize(vals, [i/10 for i in range(11)])-1)/10
        #vals = np.histogram(vals, bins=[i/10 for i in range(11)])
        plt.title(met)
        plt.hist(vals, bins=[i/10 for i in range(11)]);
    plt.tight_layout()
    plt.show()

In [ ]:
for conf in configs:
    specials = {k: v for k, v in conf.items() if isinstance(print_cnf[k], list)}
    display(Markdown("## "+", ".join(f"{k}: {v}" for k,v in specials.items())))
    ctx = SnakeContext.loader_context(config=conf)
    ctx.load("clusters")
    show_metrics(ctx.obj["clusters"]["metrics"])